In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf

In [2]:
import os
import time
import datetime
import sys   
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import shutil

In [3]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

In [4]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
spark = SparkSession.builder \
   .master("local") \
   .appName("beginner") \
   .config(conf=conf) \
   .getOrCreate()

In [5]:
sc = spark.sparkContext
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'beginner'),
 ('spark.driver.port', '43663'),
 ('spark.cores.max', '3'),
 ('spark.driver.host', '192.168.0.141'),
 ('spark.app.id', 'local-1530794262852'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.memory', '8g'),
 ('spark.executor.cores', '3'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [76]:
def fillNAN(df):
    df = df.fillna('X', subset=['flag_fthb', 'occpy_sts','channel','ppmt_pnlty',
                           'prod_type','loan_purpose','flag_sc'])
    df = df.fillna('999',subset=['mi_pct','cltv','dti','ltv'])
    df = df.fillna('9999',subset=['fico'])
    df = df.fillna('0',subset=['cd_msa'])
    df = df.fillna('99',subset=['cnt_units','cnt_borr'])
    df = df.fillna('XX',subset=['prop_type'])
    df = df.fillna('999999',subset=['zipcode']) 
    return df

In [7]:
def changedatatype(df):
    #Change the data types for all column
    for c in ['fico','cd_msa','mi_pct','cnt_units','cltv','dti','orig_upb','ltv','orig_loan_term']:
        df = df.withColumn(c,col(c).cast("double"))
    for c in ['flag_sc','flag_fthb','dt_first_pi','dt_matr','cnt_borr','occpy_sts','channel','ppmt_pnlty','zipcode','servicer_name','id_loan','loan_purpose','seller_name']:
        df = df.withColumn(c,col(c).cast("String"))
    #     df[ = df[['flag_sc','flag_fthb','cnt_borr','occpy_sts','channel','ppmt_pnlty','zipcode','servicer_name','id_loan','loan_purpose','seller_name']].astype('str')
    return df

In [8]:
def make_bins(df,in_col,out_col,splits):
    bucketizer = Bucketizer(splits=splits,inputCol=in_col, outputCol=out_col)
    df = bucketizer.setHandleInvalid("keep").transform(df)
    t1 = {}
    for i in range(len(splits)-1):
        t1[float(i)] = "["+str(splits[i])+","+str(splits[i+1])+")"
    # t = {0.0:"[0,600)", 1.0: "[600,680)", 2.0 : "[680,720)", 3.0: "[720,760)", 4.0: "[760,780)", 5.0: "[780,850)", 6.0: "[850,900)", 7.0: "[900,9999)",}
    udf_foo = udf(lambda x: t1[x], StringType())
    df = df.withColumn(out_col, udf_foo(out_col))
    return df

In [71]:
def createOriginationCombined(filepath):
    #print(str)
    
    writeHeader1 = True
    if "sample" in filepath:
        filename= "SampleOriginationCombined"
    else:
        filename= "HistoricalOriginationCombined"
#     if os.path.exists(filename):
#         shutil.rmtree(filename)
    files = glob.glob(filepath)
    F1 = udf(lambda x:x[:4]+"-"+x[4:]+"-1", StringType())
    F2 = udf(lambda x : '19'+x[2:4] if x[2:4]=='99' else '20'+x[2:4])
    t1 = time.time()
    df = spark.read.options(header=False,inferSchema=False).csv(files,sep='|',)
    df = df.toDF('fico','dt_first_pi','flag_fthb','dt_matr','cd_msa',"mi_pct",'cnt_units',
                     'occpy_sts','cltv','dti','orig_upb','ltv','int_rt','channel','ppmt_pnlty',
                     'prod_type','st', 'prop_type','zipcode','id_loan','loan_purpose', 
                     'orig_loan_term','cnt_borr','seller_name','servicer_name','flag_sc')
    df = fillNAN(df)
    df = changedatatype(df)
    df = df.withColumn("flag_fthb",when(df.flag_fthb=='9','X').otherwise(df.flag_fthb))
    # F1 = udf(lambda x:x[:4]+"-"+x[4:]+"-1", StringType())
    # df.withColumn("dt_first_pi", F1(col("dt_first_pi"))).select("dt_first_pi").show()
    for c in ['dt_first_pi','dt_matr']:
        df = df.withColumn(c,F1(col(c)).cast(DateType()).alias(c))
    df = make_bins(df,"fico","fico_bins",[0,600,680,720,760,780,850,900,9999])
    df = make_bins(df,"cltv","cltv_bins",[0,6,50,70,80,90,110,150,200,999])
    df = make_bins(df,"dti","dti_bins",[0,27,36,46,65,999])
    df = make_bins(df,"ltv","ltv_bins",[6,50,70,80,90,105,999])
    df = make_bins(df,"mi_pct","mi_pct_bins",[0,20,30,40,55,999])
    # F2 = udf(lambda x : '19'+x[2:4] if x[2:4]=='99' else '20'+x[2:4])
    df = df.withColumn("Year",F2("id_loan"))
        
#     df.toPandas().to_csv(file, mode='a', header=writeHeader1,index=False,encoding='utf-8')
    df.repartition(1).write.csv(filename,header=True,sep=',')
    shutil.move(glob.glob(os.getcwd()+"/"+filename+"/*.csv")[0],os.getcwd()+"/"+filename+".csv")
    shutil.rmtree(filename)
    t2 = time.time()
    print("Total time with pyspark : {}".format(t2-t1))
    return filename+".csv"

In [74]:
def main():
    ts = time.time()
    foldername= 'SampleInputFiles'
    
    sampleOrigFiles=str(os.getcwd())+"/"+foldername+"/sample_orig_*.txt"
#     samplePerfFiles=str(os.getcwd())+"/"+foldername+"/sample_svcg_*.txt"
    
    orig_file = createOriginationCombined(sampleOrigFiles)
    df11 = spark.read.options(header = True,inferSchema=True).csv(orig_file)
    print(df11.count())
    
#     per_file = createPerformanceCombined(samplePerfFiles)
    
#     orig_df = pd.read_csv(orig_file)
#     per_df = pd.read_csv(per_file,dtype={'delq_sts':'str'})
#     combined_df = orig_df.merge(per_df,on='id_loan')
#     combined_df.to_csv('combined_SF_smaple_data.csv', encoding='utf-8', index=False)
    
#     com1_df = pd.read_csv('combined_SF_smaple_data.csv')

#     orig_summary_statistic1 = com1_df.groupby("Year").apply(orig_summary_statistics).round(1)
#     performance_summary_statistic1 = com1_df.groupby("Year").apply(performance_summary_statistics).round(1)

#     orig_summary_statistic1.to_csv("sample_SF_orig_summary_Statistics.csv",index=False)
#     performance_summary_statistic1.to_csv("sample_SF_performance_summary_Statistics.csv",index=False)

In [75]:
if __name__ == '__main__':
    main()

Total time with pyspark : 14.63238263130188
912500


In [56]:
df11 = spark.read.options(header = True,inferSchema=True).csv('/home/dangar/Desktop/Pyspark tutorial/SampleOriginationCombined.csv/part-00000-9d3ce88a-2e60-4bb4-9f70-8e43b543735d-c000.csv')

In [58]:
df11.printSchema()

root
 |-- fico: double (nullable = true)
 |-- dt_first_pi: timestamp (nullable = true)
 |-- flag_fthb: string (nullable = true)
 |-- dt_matr: timestamp (nullable = true)
 |-- cd_msa: double (nullable = true)
 |-- mi_pct: double (nullable = true)
 |-- cnt_units: double (nullable = true)
 |-- occpy_sts: string (nullable = true)
 |-- cltv: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- orig_upb: double (nullable = true)
 |-- ltv: double (nullable = true)
 |-- int_rt: double (nullable = true)
 |-- channel: string (nullable = true)
 |-- ppmt_pnlty: string (nullable = true)
 |-- prod_type: string (nullable = true)
 |-- st: string (nullable = true)
 |-- prop_type: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- id_loan: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- orig_loan_term: double (nullable = true)
 |-- cnt_borr: integer (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- servicer_name: string (nullable =

In [ ]:
def fillNA(df):
    df['delq_sts'] = df['delq_sts'].fillna('XX')
    df['loan_age'] = df['loan_age'].fillna(999)
    df['mths_remng'] = df['mths_remng'].fillna('XX')
    df['repch_flag']=df['repch_flag'].fillna('X')
    df['flag_mod']=df['flag_mod'].fillna('X')
    df['cd_zero_bal']=df['cd_zero_bal'].fillna('00')
    df['dt_zero_bal']=df['dt_zero_bal'].fillna('189901')
    df['non_int_brng_upb']=df['non_int_brng_upb'].fillna(0)
    df['dt_lst_pi']=df['dt_lst_pi'].fillna('189901')
    df['mi_recoveries']=df['mi_recoveries'].fillna(0)
    df['net_sale_proceeds']=df['net_sale_proceeds'].fillna('0')
    df['non_mi_recoveries']=df['non_mi_recoveries'].fillna(0)
    df['expenses']=df['expenses'].fillna(0)
    df['legal_costs']=df['legal_costs'].fillna(0)
    df['maint_pres_costs']=df['maint_pres_costs'].fillna(0)
    df['taxes_ins_costs']=df['taxes_ins_costs'].fillna(0)
    df['misc_costs']=df['misc_costs'].fillna(0)
    df['actual_loss']=df['actual_loss'].fillna(0)
    df['modcost']=df['modcost'].fillna(0)
    df['stepmod_ind']=df['stepmod_ind'].fillna('X')
    return df

In [ ]:
def chnge_code_zero(x):
    if x=='00':
        return 'C'
    elif x=='01':
        return 'P'
    elif x=='06':
        return 'R'
    elif x=='03':
        return 'S'
    elif x=='09':
        return 'F'
def chnge_delinquecy(x):
    if x=='0':
        return 'C'
    elif x not in list(map(str,list(range(1,9))))+['R']:
        return '9+'
    else:
        return x

In [ ]:
t1 = time.time()
writeHeader2=True
perf_df = pd.read_csv("SampleInputFiles/sample_svcg_2011.txt",sep="|",header=None,skipinitialspace=True,dtype='unicode')
perf_df.columns =['id_loan','svcg_cycle','current_upb','delq_sts','loan_age','mths_remng', 'repch_flag',
                  'flag_mod','cd_zero_bal', 'dt_zero_bal','current_int_rt','non_int_brng_upb','dt_lst_pi',
                  'mi_recoveries','net_sale_proceeds','non_mi_recoveries','expenses', 'legal_costs', 
                  'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss', 'modcost','stepmod_ind']
#             perf_df['delq_sts'] = [ 999 if x=='R' else x for x in (perf_df['delq_sts'].apply(lambda x: x))]
#             perf_df['delq_sts'] = [ 0 if x=='XX' else x for x in (perf_df['delq_sts'].apply(lambda x: x))]
perf_df.loc[(perf_df.net_sale_proceeds=='U')|(perf_df.net_sale_proceeds=='C'),'net_sale_proceeds'] = '0'
#             perf_df['net_sale_proceeds'] = [ '0.0' if x=='C' else x for x in (perf_df['net_sale_proceeds'].apply(lambda x: x))]

#             perf_df.cd_zero_bal = perf_df.cd_zero_bal.apply(lambda x : chnge_code_zero(x))

perf_df = fillNA(perf_df)
perf_df = changedtype(perf_df)
perf_df.cd_zero_bal = perf_df.cd_zero_bal.apply(lambda x : chnge_code_zero(x))

ve =perf_df.drop(perf_df[(perf_df.cd_zero_bal=='S')|(perf_df.cd_zero_bal=='F')].index)
h = ve.groupby(by='id_loan').last().reset_index()
defauled_upb = h.loc[h.id_loan.isin(perf_df[(perf_df.cd_zero_bal=='S')|(perf_df.cd_zero_bal=='F')].id_loan.values),['id_loan','current_upb']]

ve1 =perf_df.drop(perf_df[(perf_df.cd_zero_bal=='P')|(perf_df.cd_zero_bal=='R')].index)
h1 = ve1.groupby(by='id_loan').last().reset_index()
prepaid_upb = h1.loc[h1.id_loan.isin(perf_df[(perf_df.cd_zero_bal=='P')|(perf_df.cd_zero_bal=='R')].id_loan.values),['id_loan','current_upb']]

lpi = perf_df.loc[(perf_df.delq_sts=='0'),['id_loan','svcg_cycle']]
lpi =lpi.groupby(by='id_loan').last().reset_index()
delq_sts_180 = perf_df.loc[(perf_df.delq_sts=='6'),['id_loan','svcg_cycle','current_upb']]
delq_sts_180 = delq_sts_180.groupby(by='id_loan').last().reset_index()

perf_df = perf_df.groupby(by='id_loan').last().reset_index()

perf_df['delq_sts_180_date'] = '189901'
perf_df['last_payment_date'] = '189901'
perf_df['delq_sts_180_upb'] = 0
perf_df['defaulted_upb'] = 0
perf_df['prepaid_upb'] = 0
perf_df.loc[perf_df.id_loan.isin(delq_sts_180.id_loan.values),'delq_sts_180_date'] = delq_sts_180.svcg_cycle.values
perf_df.loc[perf_df.id_loan.isin(delq_sts_180.id_loan.values),'delq_sts_180_upb'] = delq_sts_180.current_upb.values
perf_df.loc[perf_df.id_loan.isin(lpi.id_loan.values),'last_payment_date'] = lpi.svcg_cycle.values
perf_df.loc[perf_df.id_loan.isin(defauled_upb.id_loan.values),'defaulted_upb'] = defauled_upb.current_upb.values
perf_df.loc[perf_df.id_loan.isin(prepaid_upb.id_loan.values),'prepaid_upb'] = prepaid_upb.current_upb.values

#             perf_df = perf_df.drop(perf_df[perf_df.cd_zero_bal=='06'].index)
#             perf_df = perf_df.drop('repch_flag',axis=1)

perf_df.dt_lst_pi = pd.to_datetime(perf_df.dt_lst_pi.astype('str').apply(lambda x: x[:4] +'/'+x[4:]))
perf_df.dt_zero_bal = pd.to_datetime(perf_df.dt_zero_bal.astype('str').apply(lambda x: x[:4] +'/'+x[4:]))
perf_df.delq_sts_180_date = pd.to_datetime(perf_df.delq_sts_180_date.astype('str').apply(lambda x: x[:4] +'/'+x[4:]))
perf_df.last_payment_date = pd.to_datetime(perf_df.last_payment_date.astype('str').apply(lambda x: x[:4] +'/'+x[4:]))

perf_df['GT_90_days_delinquecy'] = perf_df.delq_sts.values
perf_df['GT_90_days_delinquecy'] = perf_df['GT_90_days_delinquecy'].apply(lambda x: 0 if (x=='0') | (x=='1') | (x=='2')|(x=='XX')  else 1)
perf_df['default'] = perf_df.cd_zero_bal.values
perf_df['default'] = perf_df['default'].apply(lambda x: 1 if (x=='S') | (x=='F') else 0)

perf_df['prepayment']=0
perf_df.loc[(perf_df.cd_zero_bal=='P')&(perf_df.mths_remng!=0),'prepayment'] = 1 
#             de = perf_df[perf_df.default==1]
#             months_delinquecny = (pd.to_datetime(de.dt_zero_bal.values).year - pd.to_datetime(de.last_payment_date.values).year)*12 + (pd.to_datetime(de.dt_zero_bal.values).month - pd.to_datetime(de.last_payment_date.values).month)

perf_df['lpi2zero'] = 0
perf_df['delinquent_interest'] = 0
perf_df['net_loss'] = perf_df.actual_loss.copy()
perf_df['loss_severity'] = 0

c = perf_df[(perf_df.dt_lst_pi!='1899-01-01')&(perf_df.dt_zero_bal!='1899-01-01')]
if c.shape[0]>0:
    o = (pd.to_datetime(c.dt_zero_bal.values).year - pd.to_datetime(c.dt_lst_pi.values).year)*12 + (pd.to_datetime(c.dt_zero_bal.values).month - pd.to_datetime(c.dt_lst_pi.values).month)

    perf_df.loc[(perf_df.dt_lst_pi!='1899-01-01')&(perf_df.dt_zero_bal!='1899-01-01'),'lpi2zero'] = o

    de_i = perf_df.loc[(perf_df.lpi2zero!=0)&(perf_df.default==1)] 
    perf_df.loc[(perf_df.lpi2zero!=0)&(perf_df.default==1),'delinquent_interest'] = (de_i.lpi2zero) * (de_i.defaulted_upb - de_i.non_int_brng_upb) * (de_i.current_int_rt - 0.35) / 1200

perf_df['total_proceeds'] = perf_df[['mi_recoveries','net_sale_proceeds', 'non_mi_recoveries']].sum(axis=1)

if perf_df[perf_df.actual_loss!=0].shape[0]>0:
    perf_df.loc[(perf_df.actual_loss!=0),'net_loss'] = perf_df.loc[(perf_df.actual_loss!=0),['actual_loss','modcost']].T.apply(lambda x: x[0]-x[1])

perf_df.loc[perf_df.net_loss!=0,'loss_severity'] = perf_df.loc[perf_df.net_loss!=0,['defaulted_upb','net_loss']].T.apply(lambda x: x[1]/x[0])

perf_df.delq_sts = perf_df.delq_sts.apply(lambda x : chnge_delinquecy(x))
# perf_df.to_csv(file, mode='a', header=writeHeader2,index=False,encoding='utf-8')
# writeHeader2=False
t2 = time.time()
print("Total time with pandas : {}".format(t2-t1))

In [77]:
def fillNA(df):
    df = df.fillna('X', subset=['repch_flag','flag_mod','stepmod_ind'])
    df = df.fillna('999',subset=['loan_age'])
    df = df.fillna('0',subset=['non_int_brng_upb','mi_recoveries','net_sale_proceeds',
                               'non_mi_recoveries','expenses','legal_costs','maint_pres_costs',
                               'taxes_ins_costs','misc_costs','actual_loss','modcost'])
    df = df.fillna('99',subset=['cnt_units'])
    df = df.fillna('XX',subset=['delq_sts','mths_remng'])
    df = df.fillna('189901',subset=['dt_zero_bal','dt_lst_pi'])
    df = df.fillna('00',subset=['cd_zero_bal']) 
    return df

In [ ]:
def changedtype(df):
    #Change the data types for all column
    df[['current_upb','loan_age','mths_remng','current_int_rt','non_int_brng_upb','mi_recoveries','net_sale_proceeds','non_mi_recoveries','expenses', 'legal_costs',
    'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss', 'modcost']] = df[['current_upb','loan_age','mths_remng','current_int_rt','non_int_brng_upb','mi_recoveries','net_sale_proceeds','non_mi_recoveries','expenses', 'legal_costs',
    'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss', 'modcost']].astype('float64')
    df[['id_loan','svcg_cycle','delq_sts','repch_flag','flag_mod', 'cd_zero_bal']] = df[['id_loan','svcg_cycle','delq_sts','repch_flag','flag_mod', 'cd_zero_bal']].astype('str')
    return df

In [78]:
def changedtype(df):
    #Change the data types for all column
    for c in ['current_upb','loan_age','mths_remng','current_int_rt','non_int_brng_upb','mi_recoveries','net_sale_proceeds','non_mi_recoveries','expenses', 'legal_costs',
    'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss', 'modcost']:
        df = df.withColumn(c,col(c).cast("double"))
    for x in ['id_loan','svcg_cycle','delq_sts','repch_flag','flag_mod', 'cd_zero_bal']:
        df = df.withColumn(x,col(x).cast("String"))
    #     df[ = df[['flag_sc','flag_fthb','cnt_borr','occpy_sts','channel','ppmt_pnlty','zipcode','servicer_name','id_loan','loan_purpose','seller_name']].astype('str')
    return df

In [79]:
t1 = time.time()
df = spark.read.options(header=False,inferSchema=False).csv("SampleInputFiles/sample_svcg_2011.txt",sep='|',)
df = df.toDF('id_loan','svcg_cycle','current_upb','delq_sts','loan_age','mths_remng', 'repch_flag',
                  'flag_mod','cd_zero_bal', 'dt_zero_bal','current_int_rt','non_int_brng_upb','dt_lst_pi',
                  'mi_recoveries','net_sale_proceeds','non_mi_recoveries','expenses', 'legal_costs', 
                  'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss', 'modcost','stepmod_ind')
df = fillNA(df)
df = changedtype(df)

# df.toPandas().to_csv('new.csv', mode='a', header=True,index=False,encoding='utf-8')
t2 = time.time()
print("Total time with pyspark : {}".format(t2-t1))

Total time with pyspark : 2.0707733631134033


In [80]:
df.show(5)

+------------+----------+-----------+--------+--------+----------+----------+--------+-----------+-----------+--------------+----------------+---------+-------------+-----------------+-----------------+--------+-----------+----------------+---------------+----------+-----------+-------+-----------+
|     id_loan|svcg_cycle|current_upb|delq_sts|loan_age|mths_remng|repch_flag|flag_mod|cd_zero_bal|dt_zero_bal|current_int_rt|non_int_brng_upb|dt_lst_pi|mi_recoveries|net_sale_proceeds|non_mi_recoveries|expenses|legal_costs|maint_pres_costs|taxes_ins_costs|misc_costs|actual_loss|modcost|stepmod_ind|
+------------+----------+-----------+--------+--------+----------+----------+--------+-----------+-----------+--------------+----------------+---------+-------------+-----------------+-----------------+--------+-----------+----------------+---------------+----------+-----------+-------+-----------+
|F111Q1000019|    201103|   123000.0|       0|     0.0|     360.0|         X|       X|         00|  

In [ ]:
def fillNA(df):
    df['delq_sts'] = df['delq_sts'].fillna('XX')
    df['loan_age'] = df['loan_age'].fillna(999)
    df['mths_remng'] = df['mths_remng'].fillna('XX')
    df['repch_flag']=df['repch_flag'].fillna('X')
    df['flag_mod']=df['flag_mod'].fillna('X')
    df['cd_zero_bal']=df['cd_zero_bal'].fillna('00')
    df['dt_zero_bal']=df['dt_zero_bal'].fillna('189901')
    df['non_int_brng_upb']=df['non_int_brng_upb'].fillna(0)
    df['dt_lst_pi']=df['dt_lst_pi'].fillna('189901')
    df['mi_recoveries']=df['mi_recoveries'].fillna(0)
    df['net_sale_proceeds']=df['net_sale_proceeds'].fillna('0')
    df['non_mi_recoveries']=df['non_mi_recoveries'].fillna(0)
    df['expenses']=df['expenses'].fillna(0)
    df['legal_costs']=df['legal_costs'].fillna(0)
    df['maint_pres_costs']=df['maint_pres_costs'].fillna(0)
    df['taxes_ins_costs']=df['taxes_ins_costs'].fillna(0)
    df['misc_costs']=df['misc_costs'].fillna(0)
    df['actual_loss']=df['actual_loss'].fillna(0)
    df['modcost']=df['modcost'].fillna(0)
    df['stepmod_ind']=df['stepmod_ind'].fillna('X')
    return df

In [ ]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).toPandas().T

In [ ]:
df.printSchema()

In [ ]:
df.count(),len(df.columns)

In [ ]:
v=glob.glob("SampleInputFiles/sample_orig_*.txt")

In [ ]:
df1.isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape , df1.shape

In [ ]:
df1 = pd.read_csv("SampleOriginationCombined1.csv")

In [ ]:
df = pd.read_csv("SampleOriginationCombined.csv")